In [3]:
import tensorflow as tf
import numpy as np

In [2]:
emb_encoder=a1
print(emb_encoder.shape.as_list())


encoder_outputs=[emb_encoder]
PAD_emb=tf.nn.embedding_lookup(emb,[1])

for enc_layer in range(6):
    with tf.variable_scope("encoder_%d"%enc_layer):
        emb_encoder=tf.pad(emb_encoder,paddings=[[0,0],[1,1],[0,0]],constant_values=PAD_emb)
        print(emb_encoder.shape.as_list())
        encoder_shape=emb_encoder.shape.as_list()

        emb_encoder=tf.reshape(emb_encoder,shape=[encoder_shape[0],encoder_shape[1],encoder_shape[2],1])# batch_size,seq_len,emb_dim,1

        filtersize=[3,10,1,20]
        filter=tf.Variable(initial_value=tf.truncated_normal(filtersize,stddev=0.01),name="filter")
        
        emb_encoder=tf.nn.conv2d(emb_encoder,filter=filter,strides=[1,1,1,1],padding="VALID",dilations=[1,1,1,1])
        emb_encoder=tf.reshape(emb_encoder,shape=[6,-1,10*2])
        A,B=tf.split(emb_encoder,2,axis=2)
        attn=tf.multiply(A,tf.nn.softmax(B))

        print(attn.shape.as_list())

        emb_encoder=attn+encoder_outputs[-1]
        a=tf.nn.softmax(emb_encoder,axis=1)
        print("a",a.shape.as_list())
        encoder_outputs.append(emb_encoder)
        print(emb_encoder.shape.as_list())



NameError: name 'a1' is not defined

In [6]:
a=np.array(range(48))
a=a.reshape([6,8])
a=tf.constant(a,dtype=tf.float32)
b=tf.constant([1,0,1,1,0,0],dtype=tf.float32,shape=[6,1])
g=tf.constant([[1,0,1,1,0,0],[1,0,1,1,0,0]],dtype=tf.float32,shape=[2,6])

c=tf.multiply(b,a)
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
d,f=sess.run([c,tf.nn.l2_normalize(g,1)])
print(d)
print(f)

[[  0.   1.   2.   3.   4.   5.   6.   7.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [ 16.  17.  18.  19.  20.  21.  22.  23.]
 [ 24.  25.  26.  27.  28.  29.  30.  31.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]]
[[ 0.57735026  0.          0.57735026  0.57735026  0.          0.        ]
 [ 0.57735026  0.          0.57735026  0.57735026  0.          0.        ]]


In [10]:
import tensorflow as tf 
a = tf.constant(1)
n = tf.constant(10)

def cond(a, n):
    return  a< n
def body(a,n):
    a = a + 1
    return a,n

a, n = tf.while_loop(cond, body, [a, n])
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    res = sess.run([a,n])
    print(res)


[10, 10]


In [ ]:

a=tf.constant([[123,34],[12,45],[56,23],[23,45],[65,13]],dtype=tf.int32)
b=tf.constant([[5],[7],[3],[1],[6]],tf.int32)
tf.global_variables_initializer()
sess=tf.Session()
v=sess.run([a*b])
print(v)


[array([[615, 170],
       [ 84, 315],
       [168,  69],
       [ 23,  45],
       [390,  78]], dtype=int32)]


In [7]:
class A():
    def __init__(self,a,b):
        self.a=a
        self.b=b
    
    def __call__(self, rouge):
        if "a"==rouge:
            return self.a+self.b
        elif "b" ==rouge:
            return self.a*self.b

a=A(3,2)
print(a("a"))


5


In [43]:
a=tf.Variable([])
table = tf.TensorArray(tf.int32, 12, clear_after_read=False, element_shape=[])
table=table.write(1,5)
table = tf.reshape(table.stack(), [3, 4])
print(table)
value=table[0,1]
value=tf.reshape(value,shape=[1])
value=tf.cast(value,tf.float32)
print(value)
a=tf.concat([a,value],0)
a=tf.concat([a,value],0)
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
print(sess.run(a))

Tensor("Reshape_17:0", shape=(3, 4), dtype=int32)
Tensor("Cast_7:0", shape=(1,), dtype=float32)
[5. 5.]


In [8]:
#词汇表中对应的主题词
a=tf.constant([[11,1],[12,2],[13,3],[14,4],[15,5],[16,6]],dtype=tf.int32)
# 词汇表
# b=tf.constant([[1,11,13],[3,16,6],[5,6,2]],dtype=tf.int32)
b=tf.constant([[0,13],[1,16],[0,8]],dtype=tf.int32)

emb=tf.get_variable(name='embedding',shape=[20,5],dtype=tf.float32)
topic_emb=tf.get_variable(name='topics',shape=[10,5],dtype=tf.float32)

In [16]:
a_shape=a.get_shape()[0]
b_shape=b.get_shape()
size=b_shape[0]*b_shape[1]

def look_step(g,index,single,k1):
    t=tf.cond(tf.equal(single,a[k1,0]),
            lambda :(True,k1),
            lambda :(False,-1))
    return tf.logical_or(g,t[0]),tf.maximum(index,t[1]),single,k1+1
    
def _look(single):
    g=False
    k1=0
    index=-1
    g,index,*_=tf.while_loop(
        cond=lambda g,index,single,k1:k1<a_shape,
        body=look_step,
        loop_vars=[g,index,single,k1]
    )
    return g,index


def loop_step(k,topic,matrix):
    g,index=_look(topic[k])
    matr=tf.cond(g,
                lambda :tf.gather(topic_emb,[a[index,1]]),
                lambda :tf.gather(emb,[topic[k]]))
    matr=tf.reshape(matr,[1,1,-1])
    # matrix=tf.concat([matrix,matr],0)
    matrix=matrix.write(k,matr)
    print(matrix)
    return k+1,topic,matrix

k=tf.constant(0)
g=tf.reshape(b,[-1])
# matrix=tf.Variable([[]])
matrix=tf.nn.embedding_lookup(emb,[0])
matrix=tf.reshape(matrix,[1,1,-1])
loop_vars=[tf.constant(0),tf.reshape(b,[-1]),tf.TensorArray(tf.float32,size=size)]
_,_,matrix1=tf.while_loop(
    cond=lambda k,b,*_:k<size,
    body=loop_step,
    loop_vars=loop_vars,
    # shape_invariants=[k.get_shape(),tf.TensorShape([None]),tf.TensorShape([None,1,5])]

)

matrix1=tf.reshape(matrix1.stack(),shape=[-1,b_shape[1],5])
print(matrix1)
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

matrix1,topics,emm=sess.run([matrix1,topic_emb,emb])
print(np.array(matrix1).shape)
print(matrix1)

print('------')
print(topics)
print('------')
print(emm)

Tensor("Reshape_22:0", shape=(?, 2, 5), dtype=float32)
(3, 2, 5)
[[[ 0.35722104  0.45313326  0.12768927 -0.16868928 -0.09986734]
  [-0.55134577  0.4147771  -0.10022849 -0.6070153   0.56907314]]

 [[ 0.46408054 -0.4238541   0.36293575 -0.12446699 -0.10133284]
  [ 0.5513373   0.34395307 -0.38107568  0.4736032   0.36719465]]

 [[ 0.35722104  0.45313326  0.12768927 -0.16868928 -0.09986734]
  [ 0.15290251  0.23949358 -0.0418323   0.2823467  -0.29186422]]]
------
[[-0.34714526  0.5505025   0.37950164  0.07570928 -0.26319355]
 [-0.31659645  0.5465613   0.38790613  0.34655297 -0.31055024]
 [-0.5464219   0.5692685   0.53653044 -0.18863422  0.17824364]
 [-0.55134577  0.4147771  -0.10022849 -0.6070153   0.56907314]
 [ 0.14669704 -0.01556051 -0.5932383  -0.5241651   0.513138  ]
 [-0.26796243  0.29083097 -0.07593936  0.43689567 -0.54201716]
 [ 0.5513373   0.34395307 -0.38107568  0.4736032   0.36719465]
 [-0.2004965   0.62154144  0.19659013 -0.10501605  0.39323908]
 [ 0.40405625 -0.5109581   0.45144

In [46]:
i = tf.constant(0)
l = tf.Variable([])
array = tf.Variable(tf.random_normal([10]))
def body(i, l):                                               
    temp = tf.gather(array,i)
    l = tf.concat([l, [temp]], 0)
    return i+2, l
def cond(i,l):
   return i < 10
index, list_vals = tf.while_loop(cond, body, [i, l],shape_invariants=[i.get_shape(),
                                                   tf.TensorShape([None])])
sess = tf.Session()
sess.run(tf.global_variables_initializer())
a1,b1=sess.run([list_vals,array])
print(a1)
print(b1)

[-0.12611371  0.08051919  2.6596558  -2.5261319   1.4858145 ]
[-0.12611371 -1.0288202   0.08051919 -0.40016657  2.6596558   0.4868604
 -2.5261319   0.5280155   1.4858145  -0.3094576 ]


In [47]:
gh1=tf.gather(emb,[1])
gh2=tf.gather(emb,[0])
gh=tf.concat([gh2,gh1],axis=0)
print(gh)
a1,b1=sess.run([emb,gh])

Tensor("concat_18:0", shape=(2, 5), dtype=float32)


In [20]:
t = tf.constant([[[1, 1, 1], [2, 2, 2], [7, 7, 7]],
                 [[3, 3, 3], [4, 4, 4], [8, 8, 8]],
                 [[5, 5, 5], [6, 12, 6], [9, 13, 9]],
                 [[5,12, 5], [6, 6, 6], [9, 9, 9]]])

# z1 = tf.strided_slice(t, [1], [-1], [1])
z2 = tf.strided_slice(t, [0, 2], [4, 3], [1, 1])
z3 = tf.strided_slice(t, [1, 0, 1], [-1, 2, 3], [1, 1, 1])
z4=tf.expand_dims(z2[:,-1,:],1)
print(t)
print(z4)
with tf.Session() as sess:
    # print(sess.run(z1))
    print(sess.run(z2))
    print(sess.run(z4))


Tensor("Const_17:0", shape=(4, 3, 3), dtype=int32)
Tensor("ExpandDims:0", shape=(4, 1, 3), dtype=int32)
[[[ 7  7  7]]

 [[ 8  8  8]]

 [[ 9 13  9]]

 [[ 9  9  9]]]
[[[ 7  7  7]]

 [[ 8  8  8]]

 [[ 9 13  9]]

 [[ 9  9  9]]]


In [26]:
s=np.array(range(60))

a=tf.constant(s,dtype=tf.float32,shape=[3,4,5])
c=tf.reduce_max(a,-1)
print(c)
d=tf.argmax(a,-1)
init=tf.global_variables_initializer()
sess=tf.Session()
print(sess.run([c,d]))


Tensor("Max_1:0", shape=(3, 4), dtype=float32)
[array([[ 4.,  9., 14., 19.],
       [24., 29., 34., 39.],
       [44., 49., 54., 59.]], dtype=float32), array([[4, 4, 4, 4],
       [4, 4, 4, 4],
       [4, 4, 4, 4]])]


In [25]:
i0 = tf.constant(0)
m0 = tf.ones([3, 1])
# x = tf.get_variable(name='q', shape=(3,1), initializer=tf.zeros_initializer)
# loop = 5
# def _cond(i0, m0):
#   return tf.less(i0, loop-1)
# 
# def _res(i0, m0):
#   n = tf.ones([3, 1]) + x
#   m0 = m0.write(i0,n)
#   return i0+1, m0

# loop_vars=[tf.constant(0,tf.int32),tf.TensorArray(m0.dtype,size=loop)]
# _, m0 = tf.while_loop(
#     _cond, 
#     _res, 
#     loop_vars=loop_vars,
#     # shape_invariants=[i0.get_shape(), tf.TensorShape([None, 1])]
# )
# 
# m0=tf.reshape(m0.stack(),[-1,1])
# opt = tf.train.AdagradOptimizer(1)
# 
# sess=tf.Session()
# grad = opt.compute_gradients(m0)
# sess.run(tf.global_variables_initializer())
# 
# print(sess.run(grad))

def mystack(x, n):
    loop_vars = [
        tf.constant(0, tf.int32),
        tf.TensorArray(x.dtype, size=n),
    ]
    _, fx = tf.while_loop(
        lambda j, _: j < n,
        lambda j, result: (j + 1, result.write(j, 1+x)),
        loop_vars= loop_vars
    )
    return tf.reshape(fx.stack(), (-1,1) )
 
x = tf.constant(np.random.randn(3,1), tf.float32 )
print(x)
loop = 5
m = mystack(x,loop)
print(m)
 
with tf.Session() as sess:
    print(sess.run([m,x]))

Tensor("Const_41:0", shape=(3, 1), dtype=float32)
Tensor("Reshape_25:0", shape=(?, 1), dtype=float32)
[array([[ 0.84428495],
       [ 0.5440706 ],
       [-0.28402066],
       [ 0.84428495],
       [ 0.5440706 ],
       [-0.28402066],
       [ 0.84428495],
       [ 0.5440706 ],
       [-0.28402066],
       [ 0.84428495],
       [ 0.5440706 ],
       [-0.28402066],
       [ 0.84428495],
       [ 0.5440706 ],
       [-0.28402066]], dtype=float32), array([[-0.15571503],
       [-0.45592943],
       [-1.2840207 ]], dtype=float32)]
